In [3]:
import keras
import numpy as np
import pandas as pd
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature

### Load dataset from raw.githubusercontent.com

In [6]:
data = pd.read_csv('https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv', sep=';')
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


### Split the data into training, validation and test sets

In [7]:
train, test = train_test_split(data, test_size=0.2, random_state=42)
train.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
4665,7.3,0.17,0.36,8.20,0.028,44.0,111.0,0.99272,3.14,0.41,12.4,6
1943,6.3,0.25,0.44,11.60,0.041,48.0,195.0,0.99680,3.18,0.52,9.5,5
3399,5.6,0.32,0.33,7.40,0.037,25.0,95.0,0.99268,3.25,0.49,11.1,6
843,6.9,0.19,0.35,1.70,0.036,33.0,101.0,0.99315,3.21,0.54,10.8,7
2580,7.7,0.30,0.26,18.95,0.053,36.0,174.0,0.99976,3.20,0.50,10.4,5


In [11]:
train_x = train.drop(columns=['quality'], axis=1).values
train_y = train['quality'].values.ravel()
test_x = test.drop(columns=['quality'], axis=1).values
test_y = test['quality'].values.ravel()

In [12]:
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

In [13]:
signature = infer_signature(train_x, train_y)
signature

inputs: 
  [Tensor('float64', (-1, 11))]
outputs: 
  [Tensor('int64', (-1,))]
params: 
  None

In [ ]:
def train_model(params, epochs, train_x, train_y, val_x, val_y, test_x, test_y):
    mean = np.mean(train_x, axis=0)
    var = np.var(train_x, axis=0)

    model = keras.Sequential(
        [
            keras.Input(shape=[train_x.shape[1]]), 
            keras.layers.Normalization(mean=mean, variance=var), 
            keras.layers.Dense(units=64, activation='relu'),
            keras.layers.Dense(units=1)
        ]
    )

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=params['learning_rate']), loss='mse', metrics=[keras.metrics.MeanSquaredError()])

    with mlflow.start_run(nested=True):
        model.fit(train_x, train_y, validation_data=(val_x, val_y), epochs=epochs, batch_size=params['batch_size'], verbose=0)

        eval_result = model.evaluate(test_x, test_y)

        eval_rmse = eval_result[1]

        mlflow.log_params(params)
        mlflow.log_metric('rmse', eval_rmse)
        mlflow.tensorflow.log_model(model, artifact_path='model', signature=signature, input_example=train_x[:5])

        return {'loss': eval_rmse, 'status': STATUS_OK, 'model': model}

In [15]:
def objective(params):
    result = train_model(params=params, epochs=3, train_x=train_x, train_y=train_y, val_x=val_x, val_y=val_y, test_x=test_x, test_y=test_y)

    return result

In [16]:
space = {
    'learning_rate': hp.loguniform('learning_rate', np.log(0.0001), np.log(0.1)),
    'batch_size': hp.choice('batch_size', [16, 32, 64, 128])
}

In [18]:
mlflow.set_experiment('hyperopt_keras_wine')

with mlflow.start_run():
    trials = Trials()
    best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=5, trials=trials)

    best_run = sorted(trials.results, key=lambda x: x['loss'])[0]

    mlflow.log_params(best)
    mlflow.log_metric('best_rmse', best_run['loss'])
    mlflow.tensorflow.log_model(best_run['model'], artifact_path='best_model', signature=signature)

    print('Best parameters:', best)
    print('Best RMSE:', best_run['loss'])

  0%|          | 0/5 [00:00<?, ?trial/s, best loss=?]

2025-09-25 19:17:16.536448: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2025-09-25 19:17:16.537720: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2025-09-25 19:17:16.537976: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2025-09-25 19:17:16.538274: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-09-25 19:17:16.538934: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2025-09-25 19:17:18.569423: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


 1/31 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.9444 - mean_squared_error: 0.9444
13/31 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.7290 - mean_squared_error: 0.7290 
27/31 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.6954 - mean_squared_error: 0.6954
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.6653 - mean_squared_error: 0.6653

  0%|          | 0/5 [00:06<?, ?trial/s, best loss=?]

2025/09/25 19:17:23 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 186ms/step

 1/31 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 28.9734 - mean_squared_error: 28.9734
13/31 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 28.2002 - mean_squared_error: 28.2002 
25/31 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 28.1504 - mean_squared_error: 28.1504
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 28.1149 - mean_squared_error: 28.1149

 20%|██        | 1/5 [00:20<01:05, 16.39s/trial, best loss: 0.6653105616569519]

...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step        
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step        

 1/31 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.9970 - mean_squared_error: 0.9970
13/31 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.6355 - mean_squared_error: 0.6355 
25/31 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.6310 - mean_squared_error: 0.6310
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.6287 - mean_squared_error: 0.6287
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.6101 - mean_squared_error: 0.6101

 80%|████████  | 4/5 [00:47<00:09,  9.59s/trial, best loss: 0.6073203086853027]

2025/09/25 19:18:04 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step        
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step        

100%|██████████| 5/5 [00:53<00:00, 10.69s/trial, best loss: 0.6073203086853027]

2025/09/25 19:18:09 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Best parameters: {'batch_size': 3, 'learning_rate': 0.0062816698212308205}
Best RMSE: 0.6073203086853027
